# Self-Driving Car Engineer Nanodegree
***

## Project: **Vehicle Detection** 
***

In this project, classic object detection framework, i.e., sliding window + image pyramid region proposer and HOG feature + calibrated linear SVM classifier will be implemented and used for vehicle detection.


## Set Up Session

In [ ]:
# Configuration file:
from vehicle_detection.utils.conf import Conf
# IO utilities:
import random
import glob
import matplotlib.image as mpimg
from vehicle_detection.utils.dataset import to_hdf5, read_hdf5
import pickle
# Image processing:
import numpy as np
import cv2
from vehicle_detection.extractors import ReshapeTransformer
from vehicle_detection.extractors import ColorHistogramTransformer
from vehicle_detection.extractors import HOGTransformer
from vehicle_detection.extractors import TemplateTransformer
# Visualization:
import matplotlib.pyplot as plt
%matplotlib inline

## Load Configuration

In [ ]:
conf = Conf("conf/vehicles.json")

## Explore Dataset
***

First, explore the dataset for vehicle classifier building.

In [ ]:
# Vehicle images:
vehicle_filenames = glob.glob(conf.vehicle_dataset)
# Non-vehicle images:
non_vehicle_filenames = glob.glob(conf.non_vehicle_dataset)

In [ ]:
# Vehicle sample:
vehicle_image_sample = mpimg.imread(
    random.choice(
        vehicle_filenames
    )
)
# Non-vehicle sample:
non_vehicle_image_sample = mpimg.imread(
    random.choice(
        non_vehicle_filenames
    )
)

In [ ]:
# Dataset samples:
dataset_samples_demo = plt.figure(figsize=(4, 3))
# Vehicle sample:
ax=dataset_samples_demo.add_subplot(1,2,1)
plt.imshow(vehicle_image_sample)
ax.set_title('Vehicle')
# Non-vehicle sample
ax=dataset_samples_demo.add_subplot(1,2,2)
plt.imshow(non_vehicle_image_sample)
ax.set_title('Non-Vehicle')

After viewing samples from the dataset, we know that **all the cars in images have been clearly segmented**. Thus **HOG features can be extracted directly from input image**.

We still need to **set the window size for HOG extractor**. Besides, dataset composition should also be evaluated(e.g., whether the dataset is imbalanced) so as to select the proper algorithm for classifier building.

The two stats can be attained from the following code:

In [ ]:
# Vehicle image stats:
print(
    "[  Vehicle Images  ]: Num--{}, Dimensions--{}".format(
        len(vehicle_filenames),
        np.array(
            [mpimg.imread(vehicle_filename).shape for vehicle_filename in vehicle_filenames]
        ).mean(axis = 0)
    )
)
# Non-vehicle image stats:
print(
    "[Non-Vehicle Images]: Num--{}, Dimensions--{}".format(
        len(non_vehicle_filenames),
        np.array(
            [mpimg.imread(non_vehicle_filename).shape for non_vehicle_filename in non_vehicle_filenames]
        ).mean(axis = 0)
    )
)

From the above output we know that:

**1. Window size for HOG extractor should be set as 64-by-64;**

**2. There are 8792 positive images and 8968 negative images in training dataset. The dataset is approximately balanced.**

***

Next let's try to identify the best color space for vehicle & non-vehicle color feature extraction.

In [ ]:
# Set up session:
from vehicle_detection.detectors.image_processing import resize
from vehicle_detection.utils.visualization import plot_3d

In [ ]:
# Utilities for color space exploration:
def parse_conversion(color_space):
    """
    """
    if color_space == "HSV":
        return (cv2.COLOR_BGR2HSV, ("H", "S", "V"))
    elif color_space == "Lab":
        return (cv2.COLOR_BGR2Lab, ("L*", "a*", "b*"))
    else:
        return (cv2.COLOR_BGR2RGB, ("R", "G", "B"))

def plot_pixel_distribution(image_filename, color_space):
    # Read:
    image_BGR = cv2.imread(image_filename)    
    
    # Parse conversion:
    (conversion, channels) = parse_conversion(color_space)

    # Convert subsampled image to desired color space(s):
    img_RGB = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB)  # OpenCV uses BGR, matplotlib likes RGB
    img_color_space = cv2.cvtColor(image_BGR, conversion)
    colors = img_RGB / 255.  # scaled to [0, 1], only for plotting

    # Plot and show:
    plot_3d(img_color_space, colors, axis_labels=channels)
    plt.show()

def explore_pixel_distribution(vehicle_filenames, non_vehicle_filenames, color_space):
    import random
    # Vehicles:
    plot_pixel_distribution(random.choice(vehicle_filenames), color_space)
    # Non-vehicles:
    plot_pixel_distribution(random.choice(non_vehicle_filenames), color_space)

Here YUV is selected since it is a homogeneous space and L2 norm has a clear meaning as color distance

In [ ]:
explore_pixel_distribution(vehicle_filenames, non_vehicle_filenames, "YUV")

***

Below are the HOG descriptions on all three channel components on vehicle & non-vehicle image samples

In [ ]:
def extract_channel_component_and_hog_image(image):
    """ Extract channel components and corresponding hog images for visualization
    """
    from skimage.feature import hog
    
    image_yuv = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    
    channel_components = []
    hog_images = []
    
    for channel_component in cv2.split(image_yuv):
        channel_components.append(channel_component)
        _, hog_image = hog(
            channel_component,
            orientations = conf.hog_orientations,
            pixels_per_cell = conf.hog_pixels_per_cell,
            cells_per_block = conf.hog_cells_per_block,
            transform_sqrt = conf.hog_normalize,
            block_norm = conf.hog_block_norm,
            visualise = True
        )
        hog_images.append(hog_image)
    
    return (channel_components, hog_images)

In [ ]:
# Extract descriptions:
(
    vehicle_channel_components, 
    vehicle_hog_images
) = extract_channel_component_and_hog_image(vehicle_image_sample)
(
    non_vehicle_channel_components, 
    non_vehicle_hog_images
) = extract_channel_component_and_hog_image(non_vehicle_image_sample)

# Canvas for HOG demo:
hog_demo = plt.figure(figsize=(16, 9))

# Visualize:
for channel_id, (
    vehicle_channel_component,
    vehicle_hog_image,
    non_vehicle_channel_component,
    non_vehicle_hog_image
) in enumerate(
    zip(
        vehicle_channel_components, 
        vehicle_hog_images, 
        non_vehicle_channel_components, 
        non_vehicle_hog_images
    )
):
    # Channel name:
    channel_name = "YUV"[channel_id]
    
    # Vehicle sample: 
    ax=hog_demo.add_subplot(3,4,4*channel_id + 1)
    plt.imshow(vehicle_channel_component)
    ax.set_title("Vehicle--{}-Component".format(channel_name))
    # Non-vehicle sample
    ax=hog_demo.add_subplot(3,4,4*channel_id + 2)
    plt.imshow(vehicle_hog_image)
    ax.set_title("Vehicle--{}-HOG-Image".format(channel_name))
    # Vehicle sample: 
    ax=hog_demo.add_subplot(3,4,4*channel_id + 3)
    plt.imshow(non_vehicle_channel_component)
    ax.set_title("Non-Vehicle--{}-Component".format(channel_name))
    # Non-vehicle sample
    ax=hog_demo.add_subplot(3,4,4*channel_id + 4)
    plt.imshow(non_vehicle_hog_image)
    ax.set_title("Non-Vehicle--{}-HOG-Image".format(channel_name))

## Build Training Dataset

***

Build the dataset for vehicle classifier training.

The loaded images are first serialized as 1d vector to bypass CalibratedClassifierCV's integrity check

In [ ]:
# Utilities:
def downsample(
    image_filenames, 
    sampling_percentange
):
    """ Sample image files
    """
    # Down-sample:
    image_filenames = np.random.choice(
        image_filenames, 
        int(sampling_percentange * len(image_filenames))
    )
    
    return image_filenames

def load_images(
    image_filenames,
    image_size,
    augmentation=True
):
    """ Load images
    """
    features = []
    
    # Extract features:
    for image_filename in image_filenames:
        # Load and convert to grayscale:
        object_image = cv2.resize(
            cv2.imread(image_filename),
            image_size,
            interpolation = cv2.INTER_AREA
        )
        # Prepare ROIs:
        ROIs = (object_image, cv2.flip(object_image, 1)) if augmentation else (object_image,)
        # Extract features:
        for ROI in ROIs:
            features.append(ROI)
    
    return features

In [ ]:
# Should dataset be extracted:
if conf.generate_dataset:
    # Load images:
    vehicle_images = load_images(
        downsample(vehicle_filenames, sampling_percentange=conf.sampling_percentange),
        tuple(conf.hog_window_size),
        conf.augmentation
    )
    non_vehicle_images = load_images(
        downsample(non_vehicle_filenames, sampling_percentange=conf.sampling_percentange),
        tuple(conf.hog_window_size),
        conf.augmentation
    )
    # Training set:
    X_train = np.array(vehicle_images + non_vehicle_images)
    y_train = np.array([1] * len(vehicle_images) + [-1] * len(non_vehicle_images))
    indices = np.arange(len(X_train))
    np.random.shuffle(indices)
    X_train, y_train = X_train[indices], y_train[indices]
    # Shape:
    X_train = X_train.reshape(tuple(conf.shape_serialized))
    # Dataset info:
    print(X_train.shape)
    print(y_train.shape)

## Build Classifier

***

Here I choose to implement linear SVM using LinearSVC because the dimensions of training dataset,(35520, -1), is formidable. Use SVC will lead to a very slow training process.

In [ ]:
# Cross validation:
from sklearn.model_selection import StratifiedShuffleSplit
# Classifier:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
# Evaluation metric:
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
# Hyperparameter tuning:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Model 1--Linear SVC:
def get_linear_svc():
    # Model:
    model = Pipeline(
        [
            # Deserializer:
            ('des', ReshapeTransformer(conf.shape_deserialized)),
            # Feature extractor:
            ('vec', FeatureUnion(
                [
                    ("hog", HOGTransformer(
                        color_space = conf.hog_color_space,
                        shape_only = conf.hog_shape_only,
                        orientations = conf.hog_orientations,
                        pixels_per_cell = tuple(conf.hog_pixels_per_cell),
                        cells_per_block = tuple(conf.hog_cells_per_block),
                        transform_sqrt = conf.hog_normalize,
                        block_norm = str(conf.hog_block_norm)
                    )),
                ]
            )),
            # Preprocessor:
            ('scl', StandardScaler()),
            # Classifier:
            ('clf', LinearSVC(
                penalty='l2', 
                loss=conf.classifier_loss,
                C=conf.classifier_C,
                max_iter=2000
            ))
        ]
    )

    # Hyperparameters:
    params = {
        # VEC--hog:
        #"vec__hog__pixels_per_cell": ((8,8), (16, 16)),
        # CLF--learning rate:
        #"clf__loss": ("hinge", "squared_hinge"),
        # CLF--regularization:
        #"clf__penalty": ("l1", "l2")
        "clf__C": (5e-4, 1e-3)
    }
    
    return (model, params)

In [ ]:
# Model 2--XGBoost:
def get_xgboost():
    # Model:
    model = Pipeline(
        [
            # Deserializer:
            ('des', ReshapeTransformer(conf.shape_deserialized)),
            # Feature extractor:
            ('vec', FeatureUnion(
                [
                    # 2. Shape--HOG:
                    ("hog", HOGTransformer(
                        color_space = conf.hog_color_space,
                        shape_only = conf.hog_shape_only,
                        orientations = conf.hog_orientations,
                        pixels_per_cell = tuple(conf.hog_pixels_per_cell),
                        cells_per_block = tuple(conf.hog_cells_per_block),
                        transform_sqrt = conf.hog_normalize,
                        block_norm = str(conf.hog_block_norm)
                    )),
                ]
            )),
            # Preprocessor:
            ('scl', StandardScaler()),
            # Classifier:
            ('clf', XGBClassifier(
                max_depth=8, 
                learning_rate=0.1, 
                n_estimators=1024,
                nthread=4
            ))
        ]
    )

    # Hyperparameters:
    params = {
        # VEC--hog:
        #"vec__hog__pixels_per_cell": ((8,8), (16, 16)),
        # CLF--learning rate:
        #"clf__learning_rate": (0.1, 0.3),
    }
    
    return (model, params)

In [ ]:
# Model 3--Logistic regression:
def get_logistic():
    # Model:
    model = Pipeline(
        [
            # Deserializer:
            ('des', ReshapeTransformer(conf.shape_deserialized)),
            # Feature extractor:
            ('vec', FeatureUnion(
                [
                    # 2. Shape--HOG:
                    ("hog", HOGTransformer(
                        color_space = conf.hog_color_space,
                        shape_only = conf.hog_shape_only,
                        orientations = conf.hog_orientations,
                        # Optimal--(8, 8):
                        pixels_per_cell = tuple(conf.hog_pixels_per_cell),
                        cells_per_block = tuple(conf.hog_cells_per_block),
                        # Optimal--True:
                        transform_sqrt = conf.hog_normalize,
                        block_norm = str(conf.hog_block_norm)
                    )),
                ]
            )),
            # Preprocessor:
            ('scl', StandardScaler()),
            # Classifier:
            ('clf', LogisticRegression(
                penalty='l2', 
                C=1.0,
                n_jobs=4 
            ))
        ]
    )

    # Hyperparameters:
    params = {
        # VEC--hog:
        #"vec__hog__pixels_per_cell": ((8,8), (16, 16)),
        # CLF--learning rate:
        #"clf__loss": ("hinge", "squared_hinge"),
        # CLF--regularization:
        #"clf__penalty": ("l1", "l2")
        "clf__C": (1e-3, 1e-1)
    }
    
    return (model, params)

In [ ]:
# Create cross-validation sets from the training data
cv_sets_training = StratifiedShuffleSplit(
    n_splits = 3, 
    test_size = 0.20, 
    random_state = 42
).split(X_train, y_train)

# Model 1: Linear SVC
(model, params) = get_linear_svc()
# Model 2: XGBoost:
#(model, params) = get_xgboost()
# Model 3: Logistic
#(model, params) = get_logistic()

# Make an scorer object
scorer = make_scorer(accuracy_score)

# Perform grid search on the classifier using 'scorer' as the scoring method
grid_searcher = GridSearchCV(
    estimator = model,
    param_grid = params,
    scoring = scorer,
    cv = cv_sets_training,
    n_jobs = 2,
    verbose = 10
)

# Fit the grid search object to the training data and find the optimal parameters
grid_fitted = grid_searcher.fit(X_train, y_train)

# Get parameters & scores:
best_parameters, score, _ = max(grid_fitted.grid_scores_, key=lambda x: x[1])

# Display result:
print(
    "[Best Parameters]: {}\n[Best Score]: {}".format(
        best_parameters, score
    )
)

In [ ]:
print("[Train & Calibrate Best Model]: ...")
# Get the best model
best_model = grid_fitted.best_estimator_
best_model.set_params(**best_parameters)

# Train on whole dataset with best parameters and probability calibration:
best_model_calibrated = CalibratedClassifierCV(best_model, cv=3)
best_model_calibrated.fit(X_train, y_train)
print("[Train & Calibrate Best Model]: Done.")

# Save model:
with open(conf.classifier_path, 'wb') as model_pkl:
    pickle.dump(best_model_calibrated, model_pkl)

## Vehicle Detection

In [ ]:
# Set up session:
from vehicle_detection.detectors import SlidingWindowPyramidDetector
from vehicle_detection.detectors import non_maxima_suppression
from vehicle_detection.detectors import heatmap_filtering

### Create Detector:

In [ ]:
# Initialize detector:
detector = SlidingWindowPyramidDetector(
    conf
)

### Test on Static Images

In [ ]:
# Utilities:
def detect_vehicle(image, detector, heat_thresh=None):    
    # Detect:
    bounding_boxes = detector.detect(
        image
    )
    
    # Heatmap filtering:
    if not heat_thresh is None:
        bounding_boxes = heatmap_filtering(image, bounding_boxes, heat_thresh)
        
    # Draw:
    canvas = image.copy()
    for bounding_box in bounding_boxes:
        (top, bottom, left, right) = bounding_box
        cv2.rectangle(
            canvas,
            (left, top), (right, bottom),
            (0, 255, 0),
            6
        )
        
    return canvas

In [ ]:
# Set up session:
from os.path import join, basename, splitext

# Detect:
for image_filename in glob.glob(conf.test_dataset)[-1:]:
    # Load:
    image = cv2.imread(image_filename)
    
    # Detect:
    image_raw = detect_vehicle(image, detector, None)
    image_filtered = detect_vehicle(image, detector, 2)#conf.heat_thresh)
    
    # Save:
    name, ext = splitext(basename(image_filename))
    for process_type, image_processed in zip(("raw", "filtered"), (image_raw, image_filtered)):
        cv2.imwrite(
            join(
                conf.output_path, 
                "{}-{}{}".format(
                    name,
                    process_type,
                    ext
                )
            ),
            image_processed
        )
    
    print("[{}]: Done".format(name))

In [ ]:
# Detection on static images:
for file_id in range(6):
    image_detection_demo = plt.figure(figsize=(16, 9))
    # Direct detection:
    ax=image_detection_demo.add_subplot(1,2,1)
    plt.imshow(
        mpimg.imread(
            "output_images/test{}-raw.jpg".format(file_id + 1)
        )
    )
    ax.set_title("Test Case {}--Direct Detection".format(file_id + 1))    
    # Filtered detection:
    ax=image_detection_demo.add_subplot(1,2,2)
    plt.imshow(
        mpimg.imread(
            "output_images/test{}-filtered.jpg".format(file_id + 1)
        )
    )
    ax.set_title("Test Case {}--Filtered Detection".format(file_id + 1))    

## Test on Videos

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from scipy.ndimage.measurements import label
from collections import deque
from multiprocessing import Pool
from moviepy.editor import concatenate_videoclips
from IPython.display import HTML

In [ ]:
# Spatial temporal filtering demo:
def extract_bounding_boxes_and_heat_map(image):
    # Initialize heatmap:
    H, W, _ = image.shape
    heatmap = np.zeros((H, W), dtype=np.int)
    
    # Detect:
    bounding_boxes = detector.detect(
        image
    )
    
    # Process:
    for bounding_box in bounding_boxes:
        # Aggregate heat:
        (top, bottom, left, right) = bounding_box
        heatmap[top:bottom, left:right] += 1
        # Draw:
        cv2.rectangle(
            image,
            (left, top), (right, bottom),
            (0, 255, 0),
            6
        )
    
    return (image, heatmap)

In [ ]:
# Spatial temporal filtering demo:
def extract_labelled_and_external_bounding_box(image):
    # Set up session:
    from scipy.ndimage.measurements import label
    
    # Initialize heatmap:
    H, W, _ = image.shape
    heatmap = np.zeros((H, W), dtype=np.int)
    
    # Detect:
    bounding_boxes = detector.detect(
        image
    )

    # Process:
    for bounding_box in bounding_boxes:
        # Aggregate heat:
        (top, bottom, left, right) = bounding_box
        heatmap[top:bottom, left:right] += 1   
    
    # Filter:
    heatmap[heatmap <= conf.heat_thresh] = 0

    # Label it:
    labelled, num_components = label(heatmap)

    # Identify external bounding boxes:
    external_bounding_boxes = []
    for component_id in range(1, num_components + 1):
        # Find pixels with each car_number label value
        nonzero = (labelled == component_id).nonzero()
        # Identify x and y values of those pixels
        nonzero_y, nonzero_x = np.array(nonzero[0]), np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        external_bounding_boxes.append(
            (
                np.min(nonzero_y),
                np.max(nonzero_y),
                np.min(nonzero_x),
                np.max(nonzero_x)
            )
        )

    # Draw:
    for bounding_box in external_bounding_boxes:
        (top, bottom, left, right) = bounding_box
        cv2.rectangle(
            image,
            (left, top), (right, bottom),
            (0, 255, 0),
            6
        )
    
    return (image, labelled)

In [ ]:
for frame_id in range(conf.spatial_filtering_filter_len):
    # Load frame:
    frame_filename = "test_video_frames/test-video-frame-{}.jpg".format(frame_id + 1)
    frame = cv2.imread(frame_filename)
    
    # Get bounding boxes plot and heatmap:
    (boxes_image, heatmap) = extract_bounding_boxes_and_heat_map(frame)
    
    # Initialize canvas:
    spatial_temporal_filtering_demo = plt.figure(figsize=(16, 9))
    
    # Direct detection:
    ax=spatial_temporal_filtering_demo.add_subplot(1,2,1)
    plt.imshow(
        cv2.cvtColor(boxes_image, cv2.COLOR_BGR2RGB)
    )
    ax.set_title("Frame {}--Bounding Boxes".format(frame_id + 1))
    
    # Direct detection:
    ax=spatial_temporal_filtering_demo.add_subplot(1,2,2)
    plt.imshow(
        heatmap
    )
    ax.set_title("Frame {}--Heatmap".format(frame_id + 1))

In [ ]:
# Load frame:
frame_filename = "test_video_frames/test-video-frame-7.jpg"
frame = cv2.imread(frame_filename)
    
# Get bounding boxes plot and heatmap:
(boxes_image, labelled) = extract_labelled_and_external_bounding_box(frame)
    
# Initialize canvas:
spatial_temporal_filtering_demo = plt.figure(figsize=(16, 9))
    
# Direct detection:
ax=spatial_temporal_filtering_demo.add_subplot(1,2,1)
plt.imshow(
    cv2.cvtColor(boxes_image, cv2.COLOR_BGR2RGB)
)
ax.set_title("Frame 7--Filtered Bounding Boxes")
    
# Direct detection:
ax=spatial_temporal_filtering_demo.add_subplot(1,2,2)
plt.imshow(
    labelled
)
ax.set_title("Frame 7--Labelled")

In [ ]:
# Static variable decorator:
def static_vars(**kwargs):
    def decorate(func):
        for k in kwargs:
            setattr(func, k, kwargs[k])
        return func
    return decorate

# Frame processor:
@static_vars(
    TEMPORAL_FILTER_LEN=conf.spatial_filtering_filter_len,
    bounding_boxes_queue=deque(), 
    heatmap_accumulator = np.zeros(
        tuple(conf.spatial_filtering_frame_size), 
        dtype=np.int
    )
)
def process_frame(frame):
    """ Detect vehicles in given frame
    """
    # Format:
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
    # Detect:
    bounding_boxes_current = detector.detect(frame)
    
    # Spatial filtering:
    bounding_boxes_current = heatmap_filtering(
        frame, 
        bounding_boxes_current, 
        conf.heat_thresh
    )

    # Temporal filtering:
    if len(process_frame.bounding_boxes_queue) == process_frame.TEMPORAL_FILTER_LEN:
        # Remove left one:
        for bounding_box in process_frame.bounding_boxes_queue.popleft():
            (top, bottom, left, right) = bounding_box
            process_frame.heatmap_accumulator[top:bottom, left:right] -= 1
    
    # Append:
    process_frame.bounding_boxes_queue.append(bounding_boxes_current)
        
    # Aggregate heat:
    for bounding_box in bounding_boxes_current:
        (top, bottom, left, right) = bounding_box
        process_frame.heatmap_accumulator[top:bottom, left:right] += 1
    
    # Filter:
    heatmap = process_frame.heatmap_accumulator.copy()
    heat_thresh = int(0.8 * len(process_frame.bounding_boxes_queue))
    heatmap[heatmap <= heat_thresh] = 0

    # Label it:
    labelled, num_components = label(heatmap)

    # Identify external bounding boxes:
    bounding_boxes_filtered = []
    for component_id in range(1, num_components + 1):
        # Find pixels with each car_number label value
        nonzero = (labelled == component_id).nonzero()
        # Identify x and y values of those pixels
        nonzero_y, nonzero_x = np.array(nonzero[0]), np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bounding_boxes_filtered.append(
            (
                np.min(nonzero_y),
                np.max(nonzero_y),
                np.min(nonzero_x),
                np.max(nonzero_x)
            )
        )
    
    # Draw:
    for bounding_box in bounding_boxes_filtered:
        (top, bottom, left, right) = bounding_box
        cv2.rectangle(
            frame,
            (left, top), (right, bottom),
            (0, 255, 0),
            6
        )
        
    return cv2.resize(
        cv2.cvtColor(frame, cv2.COLOR_BGR2RGB),
        (960, 540)
    )    

In [ ]:
def video_process_worker(worker_id):
    # Specify input & output:
    input_filename = video_project_input
    output_filename = video_project_output.format(worker_id + 1)
    
    # Get workload:
    start, end = 10*worker_id, 10*(worker_id + 1)
    
    # Process:
    clip_project = VideoFileClip(input_filename).subclip(start, end)
    clip_project_detected = clip_project.fl_image(process_frame)
    clip_project_detected.write_videofile(output_filename, audio=False)

### Test Video, Shorter One

In [ ]:
# IO config:
video_test_input = "test_video.mp4"
video_test_output = "output_videos/test_video_detected.mp4"

In [ ]:
### Process:
clip_test = VideoFileClip(video_test_input)
clip_test_detected = clip_test.fl_image(process_frame)
%time clip_test_detected.write_videofile(video_test_output, audio=False)

In [ ]:
# Display:
HTML(
    """
    <video width="960" height="540" controls>
      <source src="{0}">
    </video>
    """.format(video_test_output)
)

### Project Video, Longer One

In [ ]:
# IO config:
video_project_input = "project_video.mp4"
video_project_output = "output_videos/project_video_detected_{}.mp4"

In [ ]:
# Process--parallel:
pool = Pool(5)
pool.map(video_process_worker, range(5))

In [ ]:
# Merge all clips:
clips = [VideoFileClip(video_project_output.format(id + 1)) for id in range(5)]
concat_clip = concatenate_videoclips(clips, method="chain")
%time concat_clip.write_videofile(video_project_output.format(0), audio=False)

In [ ]:
# Display:
HTML(
    """
    <video width="960" height="540" controls>
      <source src="{0}">
    </video>
    """.format(video_project_output)
)